# Import Libraries 

In [1]:
import pandas as pd
import numpy as np
import os
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelEncoder

# Load Dataset
En esta celda recorro todo el dataset para tomar metricas pero por una cuestion de performance y tamanho en memoria no lo guardo, ya que luego voy a cargar de a uno para entrenar el clasificador.

In [2]:
dataset_home = 'datasets/haskell/'

df = pd.DataFrame(
    columns=['course', 'submission_amount', 'errored', 'failed', 'passed', 'passed_with_warnings'])

for course in os.listdir('datasets/haskell/'):
    json_file = open(os.path.join(dataset_home, course), encoding='utf8')
    json_data = json.load(json_file)
    submissions = pd.io.json.json_normalize(json_data)
    submissions = submissions[~(submissions['status'] == 'aborted')]
    submissions = submissions[~(submissions['status'] == 'pending')]
    submissions = submissions[~(submissions['status'] == 'running')]
    submissions = submissions[~(submissions['status'] == 'manual_evaluation_pending')]
    
    total_amount_submissions = submissions.shape[0]
    submissions_grouped = submissions.groupby(['status']).size()
    metrics = {}
    metrics = submissions_grouped / total_amount_submissions
    metrics['course'] = str(course)
    metrics['submission_amount'] = total_amount_submissions
    df = df.append(metrics, ignore_index=True)

# Draft Annotations
Descartamos los ejercicios con menos de 100 submissions porque no son significativos para lo que queremos hacer. Por otro lado, a las submissions le sacamos los aborted, pending and manual_evaluation_pending, porque tampoco aportan

Aca hay cosas para decidir tenemos actividades como 
223 	submissions_¡Terminala! Parte 1.json 	309 	NaN 	NaN 	1.000000 	NaN 	NaN
la cual es simplemente un click lo que hacen y pasan el ejercicio, por el momento los tire del dataset 

In [3]:
short_df = df[df['submission_amount'] > 100].fillna(value=0).sort_values('submission_amount')
short_df = short_df.where(short_df['passed'] != 1).dropna()
mean_distribution = (short_df[["errored", "failed", "passed", "passed_with_warnings"]].astype(float).multiply(short_df["submission_amount"], axis="index")).sum()/(short_df['submission_amount']).sum()
mean_distribution['course'] = 'Weighted Mean'
mean_distribution['submission_amount'] = df['submission_amount'].sum()
short_df.loc[len(short_df)+1] = mean_distribution
short_df

,course,submission_amount,errored,failed,passed,passed_with_warnings
225,submissions_Descontrolarse.json,107.0,0.205607,0.598131,0.196262,0.000000
290,"submissions_validarIguales, sobre validados.json",120.0,0.200000,0.683333,0.083333,0.033333
249,submissions_quienesPueden.json,129.0,0.131783,0.488372,0.317829,0.062016
127,submissions_Alcohol en Sangre con fold.json,131.0,0.358779,0.366412,0.274809,0.000000
64,submissions_positivosYNegativos.json,149.0,0.201342,0.375839,0.395973,0.026846
112,submissions_Pedir bebida.json,149.0,0.161074,0.496644,0.342282,0.000000
211,submissions_Composción.json,152.0,0.105263,0.355263,0.526316,0.013158
244,submissions_validarIguales.json,165.0,0.363636,0.557576,0.072727,0.006061
82,submissions_Haciendo functores.json,166.0,0.620482,0.090361,0.289157,0.000000
91,submissions_estadisticas.json,179.0,0.256983,0.312849,0.379888,0.050279


Para cada curso en el dataframe de arriba voy a entrenar un clasificar linear, tomarle las metricas y guardarlo en un df.
Aca hay un detalle no trivial a revisar, la cantidad de passed_with_warnings suele ser tan baja que hay casos donde esta en el train set pero no el test, entonces el score no se puede calcular.


In [21]:
#define dataframe to save performance for each classfier
linear_performance = pd.DataFrame(columns=['Exercise', 'precision', 'recall', 'f1-score', 'amount_tested', 'submission_amount'])
i = 0
amount_courses = len(short_df['course'])
not_trained = []
for course in short_df['course'][:-1]:
    #Load json file
    json_file = open(os.path.join(dataset_home, course), encoding='utf8')
    json_data = json.load(json_file)
    
    #Drop submissions meaningless
    submissions = pd.io.json.json_normalize(json_data)
    submissions = submissions[submissions['content'] != ""]
    submissions = submissions[~submissions['content'].isnull()]
    submissions = submissions[~(submissions['status'] == 'aborted')]
    submissions = submissions[~(submissions['status'] == 'pending')]
    submissions = submissions[~(submissions['status'] == 'running')]
    submissions = submissions[~(submissions['status'] == 'manual_evaluation_pending')]
    
    #at least to train has to have 10 submissions
    #introduce this because exercises with empty content
    if submissions.shape[0] < 10:
        print('Exercise with less than 10 submission content not empty {}'.format(course))
        continue
    #Split dataset in train and test
    try:
        X = submissions['content']
        Y = submissions['status']
        X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.40, random_state=32)

        #Featurize submissions content
        cv = CountVectorizer(lowercase=False)
        X_transformed = cv.fit_transform(X_train)

        #Featurize status submissions
        le = LabelEncoder()
        Y_transformed = le.fit_transform(Y_train)

        #Train linear regression for multi label classification
        lr = OneVsRestClassifier(LinearRegression())
        lr.fit(X_transformed, Y_transformed)

        #classify Test set
        predicted = lr.predict(cv.transform(X_test))

        #obtain metrics avg/total
        #shape of metrics
        #precision    recall  f1-score   amount_tested
        metrics_by_exercise = classification_report(
            le.transform(Y_test), predicted, target_names=le.classes_, digits=2).split()[-4:]
        metrics_by_exercise = [str(course)] + metrics_by_exercise + [int(X_train.shape[0] + X_test.shape[0])]
        linear_performance.loc[len(linear_performance)+1]= metrics_by_exercise
        i += 1
    except:
        not_trained.append(course)
        pass
    
print("Trained {} classifier of {}".format(i, amount_courses))

/home/mrc/anaconda3/envs/mumuki/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mrc/anaconda3/envs/mumuki/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/mrc/anaconda3/envs/mumuki/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mrc/anaconda3/envs/mumuki/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samp

/home/mrc/anaconda3/envs/mumuki/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mrc/anaconda3/envs/mumuki/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mrc/anaconda3/envs/mumuki/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mrc/anaconda3/envs/mumuki/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with n

/home/mrc/anaconda3/envs/mumuki/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mrc/anaconda3/envs/mumuki/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mrc/anaconda3/envs/mumuki/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Trained 219 classifier of 226


In [26]:
mean_performance = (linear_performance[["precision", "recall", "f1-score"]].astype(float).multiply(linear_performance["submission_amount"], axis="index")).sum()/(linear_performance['submission_amount']).sum()
mean_performance['Exercise'] = "Weighted Mean"
mean_performance['amount_tested'] = linear_performance['amount_tested'].astype(int).sum()
mean_performance['submission_amount'] = linear_performance['submission_amount'].sum()
linear_performance.loc[len(linear_performance) + 1] = mean_performance

In [28]:
linear_performance

,Exercise,precision,recall,f1-score,amount_tested,submission_amount
1,submissions_Descontrolarse.json,0.50,0.49,0.49,43,107
2,"submissions_validarIguales, sobre validados.json",0.58,0.65,0.60,48,119
3,submissions_quienesPueden.json,0.63,0.58,0.59,52,129
4,submissions_Alcohol en Sangre con fold.json,0.72,0.71,0.71,52,130
5,submissions_positivosYNegativos.json,0.61,0.52,0.52,60,148
6,submissions_Pedir bebida.json,0.53,0.57,0.55,60,149
7,submissions_Composción.json,0.63,0.52,0.52,61,151
8,submissions_validarIguales.json,0.71,0.70,0.70,66,163
9,submissions_Haciendo functores.json,0.54,0.58,0.56,66,165
10,submissions_estadisticas.json,0.51,0.54,0.52,72,178


### ejercicios que no se pudieron entrenar
Como el split de train y test es random esto varia segun el seed del random

In [18]:
not_trained

['submissions_primeroQueCumple.json',
 'submissions_tipos-12.json',
 'submissions_tipos-10.json',
 'submissions_cuentaBizarra.json',
 'submissions_mitad.json',
 'submissions_notasFinales2-upla.json']